In [1]:
import import_ipynb
import spacy as sp
import json
import pandas as pd
import joblib

In [2]:
import spacy_helper_methods as sph

importing Jupyter notebook from spacy_helper_methods.ipynb


# Extract entities from patent database

In [3]:
df = pd.read_json('../preprocessed_files/patents.json')
df

,country,doc-number,date,application-reference,title,assignee,inventors,abstract,claims
0,US,20230225235,20230720,"{'country': 'US', 'doc-number': 17754513, 'dat...","AGRICULTURAL TRENCH DEPTH SYSTEMS, METHODS, AN...","{'orgname': 'Precision Planting LLC', 'city': ...","[{'last-name': 'Sloneker', 'first-name': 'Dill...",\nA row unit (10) of an agricultural planter w...,a row unit frame;\na furrow opening disc rotat...
1,US,20230225236,20230720,"{'country': 'US', 'doc-number': 18007883, 'dat...",Agricultural Attachment for Cultivating Row Crops,{'orgname': 'Amazonen-Werke H. Dreyer SE & Co....,"[{'last-name': 'RESCH', 'first-name': 'Rainer'...",\nThe invention relates to an agricultural att...,"a row-detection device adapted to detect, duri..."
2,US,20230225237,20230720,"{'country': 'US', 'doc-number': 18121636, 'dat...",TRAVEL LINE CREATION SYSTEM FOR AGRICULTURAL M...,"{'orgname': 'KUBOTA CORPORATION', 'city': 'Osa...","[{'last-name': 'MORIMOTO', 'first-name': 'Taka...",\nA travel line creation system for an agricul...,a position acquirer to acquire position measur...
3,US,20230225238,20230720,"{'country': 'US', 'doc-number': 18187398, 'dat...",AGRICULTURAL HARVESTING MACHINE WITH PRE-EMERG...,"{'orgname': 'Deere & Company', 'city': 'Moline...","[{'last-name': 'BLANK', 'first-name': 'Sebasti...",\nAn agricultural harvesting machine includes ...,crop processing functionality configured to en...
4,US,20230225239,20230720,"{'country': 'US', 'doc-number': 18190358, 'dat...","DETECTION OF PLANT DISEASES WITH MULTI-STAGE, ...","{'orgname': 'CLIMATE LLC', 'city': 'Saint Loui...","[{'last-name': 'Guan', 'first-name': 'Wei', 'c...",\nA computer system is provided comprising a c...,a classification model management server compu...
...,...,...,...,...,...,...,...,...,...
906,US,20230226142,20230720,"{'country': 'US', 'doc-number': 18166956, 'dat...",IONIC SELF-ASSEMBLING PEPTIDES,"{'orgname': '3-D MATRIX, LTD.', 'city': 'Tokyo...","[{'last-name': 'Gil', 'first-name': 'Eun Seok'...",\nProvided herein are ionic self-assembling pe...,\n\na) an N-terminal functional group selected...
907,US,20230226142,20230720,NaN,NaN,NaN,NaN,NaN,NaN
908,US,20230226143,20230720,"{'country': 'US', 'doc-number': 18011718, 'dat...",PHARMACEUTICAL COMPOSITION COMPRISING A COMBIN...,"{'orgname': 'OCVIRK, Soren', 'city': 'Kranzber...","[{'last-name': 'Ocvirk', 'first-name': 'Sören'...",\nA pharmaceutical composition includes a comb...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n
909,US,20230226144,20230720,"{'country': 'US', 'doc-number': 18186639, 'dat...",TOPICAL ANTIBIOTIC,"{'orgname': 'Concept Matrix Solutions', 'city'...","[{'last-name': 'LaRosa', 'first-name': 'Tony',...",\nProvided herein is a topical antibiotic comp...,"(a) at least one of bacitracin, bacitracin zin..."


In [4]:
print(df.shape)
df = df.dropna()
print(df.shape)

(911, 9)
(879, 9)


In [5]:
nlp = sp.load('en_core_sci_lg')

/Users/sagarjogadhenu/anaconda3/lib/python3.9/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_sci_lg' (0.5.3) was trained with spaCy v3.6.1 and may not be 100% compatible with the current version (3.7.2). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/Users/sagarjogadhenu/anaconda3/lib/python3.9/site-packages/spacy/language.py:2170: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


In [9]:
model = joblib.load('trained_tech_classifier_model.joblib')

In [7]:
def filter_non_tech(x, model):
    if x:
        y_pred = model.predict(x)
    return([x[i] for i in range(len(x)) if y_pred[i]])

def extract_tech_entities(nlp, model, ds):
    lemma_ds = sph.lemmatize(nlp, ds)
    ent_ds = sph.get_entities(nlp,lemma_ds)
    ent_ds = ent_ds.apply(lambda x: [ent[0] for ent in x])
    ent_ds = ent_ds.apply(lambda x: filter_non_tech(x, model))
    return ent_ds
        

In [ ]:
#lemma_df = sph.lemmatize(nlp, df['abstract'])
#ent_df = sph.get_entities(nlp,lemma_df)
#ent_df = ent_df.apply(lambda x: [ent[0] for ent in x])

In [10]:
%%time
df['abstract_entities'] = extract_tech_entities(nlp,model,df['abstract'])
df['claim_entities'] = extract_tech_entities(nlp, model, df['claims'])                                              

CPU times: user 1min 31s, sys: 5.75 s, total: 1min 37s
Wall time: 1min 46s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [14]:
df

,country,doc-number,date,application-reference,title,assignee,inventors,abstract,claims,abstract_entities,claim_entities
0,US,20230225235,20230720,"{'country': 'US', 'doc-number': 17754513, 'dat...","AGRICULTURAL TRENCH DEPTH SYSTEMS, METHODS, AN...","{'orgname': 'Precision Planting LLC', 'city': ...","[{'last-name': 'Sloneker', 'first-name': 'Dill...",\nA row unit (10) of an agricultural planter w...,a row unit frame;\na furrow opening disc rotat...,"[agricultural planter apparatus, depth trench ...","[open furrow soil, surface row unit, direction..."
1,US,20230225236,20230720,"{'country': 'US', 'doc-number': 18007883, 'dat...",Agricultural Attachment for Cultivating Row Crops,{'orgname': 'Amazonen-Werke H. Dreyer SE & Co....,"[{'last-name': 'RESCH', 'first-name': 'Rainer'...",\nThe invention relates to an agricultural att...,"a row-detection device adapted to detect, duri...","[row crop, row-detection device design, detect...","[row-detection device, detect, cultivation pro..."
2,US,20230225237,20230720,"{'country': 'US', 'doc-number': 18121636, 'dat...",TRAVEL LINE CREATION SYSTEM FOR AGRICULTURAL M...,"{'orgname': 'KUBOTA CORPORATION', 'city': 'Osa...","[{'last-name': 'MORIMOTO', 'first-name': 'Taka...",\nA travel line creation system for an agricul...,a position acquirer to acquire position measur...,"[agricultural machine, position, point agricul...","[position, measurement, agricultural machine, ..."
3,US,20230225238,20230720,"{'country': 'US', 'doc-number': 18187398, 'dat...",AGRICULTURAL HARVESTING MACHINE WITH PRE-EMERG...,"{'orgname': 'Deere & Company', 'city': 'Moline...","[{'last-name': 'BLANK', 'first-name': 'Sebasti...",\nAn agricultural harvesting machine includes ...,crop processing functionality configured to en...,"[crop processing, crop field, crop processing,...","[crop processing, crop field, crop processing,..."
4,US,20230225239,20230720,"{'country': 'US', 'doc-number': 18190358, 'dat...","DETECTION OF PLANT DISEASES WITH MULTI-STAGE, ...","{'orgname': 'CLIMATE LLC', 'city': 'Saint Loui...","[{'last-name': 'Guan', 'first-name': 'Wei', 'c...",\nA computer system is provided comprising a c...,a classification model management server compu...,"[computer system, classification, instruction,...","[classification, instruction, image user devic..."
...,...,...,...,...,...,...,...,...,...,...,...
904,US,20230226141,20230720,"{'country': 'US', 'doc-number': 18157933, 'dat...",ANGIOTENSIN II ALONE OR IN COMBINATION FOR THE...,{'orgname': 'The George Washington University ...,"[{'last-name': 'CHAWLA', 'first-name': 'Lakhmi...","\nThe present invention relates, inter alia, t...",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"[method, output shock, treatment, raise blood ...",[]
906,US,20230226142,20230720,"{'country': 'US', 'doc-number': 18166956, 'dat...",IONIC SELF-ASSEMBLING PEPTIDES,"{'orgname': '3-D MATRIX, LTD.', 'city': 'Tokyo...","[{'last-name': 'Gil', 'first-name': 'Eun Seok'...",\nProvided herein are ionic self-assembling pe...,\n\na) an N-terminal functional group selected...,"[ionic self-assembling peptide, pharmaceutical...","[n-terminal functional group, group, 7-methoxy..."
908,US,20230226143,20230720,"{'country': 'US', 'doc-number': 18011718, 'dat...",PHARMACEUTICAL COMPOSITION COMPRISING A COMBIN...,"{'orgname': 'OCVIRK, Soren', 'city': 'Kranzber...","[{'last-name': 'Ocvirk', 'first-name': 'Sören'...",\nA pharmaceutical composition includes a comb...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n,"[pharmaceutical composition, short-chain c2 c5...",[]
909,US,20230226144,20230720,"{'country': 'US', 'doc-number': 18186639, 'dat...",TOPICAL ANTIBIOTIC,"{'orgname': 'Concept Matrix Solutions', 'city'...","[{'last-name': 'LaRosa', 'first-name': 'Tony',...",\nProvided herein is a topical antibiotic comp...,"(a) at least one of bacitracin, bacitracin zin...","[antibiotic agent, method, skin surface]","[zinc, hard paraffin, thc, tetrahydrocannabino..."


In [21]:
df.to_csv('patents_entities.csv')

# Extract entities from SBIR database

In [31]:
sbir_df = pd.read_csv('award_data.csv')
len(sbir_df)

/var/folders/fj/31jlj_jj6qq7jtnhvsydh1vr0000gn/T/ipykernel_80705/1165860663.py:1: DtypeWarning: Columns (10,12,13,15,38,39,40) have mixed types. Specify dtype option on import or set low_memory=False.
  sbir_df = pd.read_csv('award_data.csv')


201856

In [32]:
sbir_df = sbir_df.sample(1000)

In [35]:
sbir_df = sbir_df[~sbir_df['Abstract'].isna()]
len(sbir_df)

871

In [36]:
%%time
sbir_df['abstract_entities'] = extract_tech_entities(nlp, model, sbir_df['Abstract'])

CPU times: user 3min 30s, sys: 28.6 s, total: 3min 59s
Wall time: 5min 55s


<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


145112    [transponder, multiple frequency band, microwa...
106926    [Tactical radio system, wideband antenna set, ...
126831    [datum fusion, decision support, decision comp...
25069     [goal phase, information, submarine, acoustic ...
83500     [current, paradigm management, traffic, voice ...
                                ...                        
3223      [energy generation, doe industry, stimulation,...
190905    [LOGISTICS MANAGEMENT, E.G. SUPPLY unit ammuni...
122365    [DESCRIPTION, applicant, fire, earthquake, tor...
104675    [SET, system, Omnispectral sensing, unmanned i...
32103     [light weight, portable power high power densi...
Name: abstract_entities, Length: 871, dtype: object

In [39]:
sbir_df.to_csv('sbir_entities.csv')